# Time Data Analyser

- Notebook que faz a analise dos dados de tempo gasto que foi coletado no codigo que roda no uControlador

+ Ha **7** colunas no arquivo csv de dados do tempo (em microsegundos):
    1. **`TempoLeituraSD`:** Mede o tempo gasto para ler 2048 amostras de audio do arquivo de entrada.
    2. **`TempoNormalizacao`:** Mede o tempo gasto para normalizar e transformar os dados em float.
    3. **`TempoPreProcessamento`:** Mede o tempo gasto para multiplicar o _sinal de entrada_ (`inputSignal`) com a _janela de Hanning_ e copiar o valor para o buffer que vai ser usado para o processamento dos dados.
    4. **`TempoFeatTempo`:** Mede o tempo gasto com a extracao das features relacionadas ao dominio do tempo.
    5. **`TempoFFT`:** Mede o tempo gasto para calcular a FFT do _sinal de entrada_.
    6. **`TempoFeatFrequencia`:** Mede o tempo gasto com a extracao das features relacionadas ao dominio da frequencia.
    7. **`TempoInferencia`:** Mede o tempo gasto para fazer a inferencia.

+ Os dados foram extraidos utilizando os modelos: **_Decision Tree_**, **_Extra Tree_**, **_Gaussian Naive Bayess_** e **_Random Forest_**
    - Alem disso, para cada modelo foi feito a captura dos dados de tempo para cada um dos arquivos de audio: _Off Condition with noise_, _Healthy condition_, _Bearing fault (F1)_, _Fan fault (F2)_ e _Gear fault (F3)_.

In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
from pathlib import Path

In [ ]:
input_file_set = {
    "off": "../data/extracted_features/uControlador/decision_tree/times/DT-T-OFF.CSV",
    "health": "../data/extracted_features/uControlador/decision_tree/times/DT-T-HTH.CSV",
    "f1": "../data/extracted_features/uControlador/decision_tree/times/DT-T-F1.CSV",
    "f2": "../data/extracted_features/uControlador/decision_tree/times/DT-T-F2.CSV",
    "f3": "../data/extracted_features/uControlador/decision_tree/times/DT-T-F3.CSV",
}

CSV_COLUMNS_NAMES=['TempoLeituraSD','TempoNormalizacao','TempoPreProcessamento','TempoFeatTempo','TempoFFT','TempoFeatFrequencia','TempoInferencia']

AUDIO_SAMPLE_RATE = 48000   # 48kHz, do arquivo de audio .wav
INPUT_BUFFER_SIZE = 2048    # Tamanho do buffer usado

## Carregando os dados

In [ ]:
FILE_SELECTOR=0
# batch_df = pd.DataFrame()
time_df = pd.read_csv(input_file_set['off'], delimiter=',', header=0)
input_filename= Path(input_file_set['off']).stem
print(f"Reading file {input_filename}")

## Processando os dados

In [ ]:
# cria coluna batch_id para ajudar a agrupar as janelas
# 4 janelas sao geradas para as 2048 amostras lidas do SD card
time_df['batch_id'] = time_df.index // 4
print(time_df.head(8))

In [ ]:
# Estatisticas por batch (janela de amostragem)
stats_per_batch = time_df.groupby('batch_id').agg(['mean', 'std', 'min', 'max'])
print(stats_per_batch.columns)
print(stats_per_batch.loc[0])

In [ ]:
# Cria uma nova coluna que e a soma de todos os tempos
# Remove a coluna TempoLeituraSD da conta
time_df['tempo_total_janela'] = time_df[['TempoNormalizacao','TempoPreProcessamento','TempoFeatTempo','TempoFFT','TempoFeatFrequencia','TempoInferencia']].sum(axis=1)

# Tempo total para processar as 4 janelas (1 batch)
time_df['tempo_total_batch'] = time_df.groupby('batch_id')['tempo_total_janela'].sum()
batch_df = pd.DataFrame(time_df.groupby('batch_id')['tempo_total_janela'].sum())
batch_df = batch_df.rename(columns={"tempo_total_janela": "tempo_total_batch"})
# batch_df = pd.DataFrame(batch_df)

# Comprar o tempo gasto no processamento com o tempo maximo para poder 
# processar em tempo real
max_time_per_batch = (INPUT_BUFFER_SIZE / AUDIO_SAMPLE_RATE) * 1e6 # aprox ~ 42666us
print(f"Tempo maximo para funcionar em tempo real eh {max_time_per_batch} us")
batch_df['batch_em_rt'] = batch_df['tempo_total_batch'] <= max_time_per_batch

# Porcentagem de batchs que poderia ter sido processados com tempo real
conformidade = batch_df.groupby('batch_id')['batch_em_rt'].all()
print(f"Batches em tempo real: {conformidade.sum()}/{len(conformidade)}")

In [ ]:
# batch_df.info()
# batch_df.head()

## Graficos

In [ ]:
import matplotlib.pyplot as plt

### Media de cada Janela por Batch

In [ ]:
stats_per_batch = stats_per_batch.transpose().reset_index()
# stats_per_batch.columns = ['Coluna', 'Média', 'Desvio Padrão', 'Mínimo', 'Máximo']
stats_per_batch.head()

In [ ]:
# Calcular tempo medio por janela em todos os batches
# mean_times = stats_per_batch.xs('mean', axis=1, level=1)

# # Plotar
# mean_times.plot(kind='bar', figsize=(15, 6))
# plt.title('Tempo Médio por Etapa (por Batch)')
# plt.ylabel('Tempo (µs)')
# plt.xlabel('Batch ID')
# plt.show()

# Configurar a figura
# plt.figure(figsize=(14, 8))
# largura_barra = 0.2  # Largura das barras
# posicoes = range(len(stats_per_batch['Coluna']))  # Posições no eixo X

# # Plotar cada estatística
# plt.bar(
#     [p - 1.5*largura_barra for p in posicoes],
#     stats_per_batch['Média'],
#     width=largura_barra,
#     label='Média',
#     color='blue'
# )

# plt.bar(
#     [p - 0.5*largura_barra for p in posicoes],
#     stats_per_batch['Desvio Padrão'],
#     width=largura_barra,
#     label='Desvio Padrão',
#     color='orange'
# )

# plt.bar(
#     [p + 0.5*largura_barra for p in posicoes],
#     stats_per_batch['Mínimo'],
#     width=largura_barra,
#     label='Mínimo',
#     color='green'
# )

# plt.bar(
#     [p + 1.5*largura_barra for p in posicoes],
#     stats_per_batch['Máximo'],
#     width=largura_barra,
#     label='Máximo',
#     color='red'
# )

# # Ajustar detalhes do gráfico
# plt.title('Estatísticas das Colunas de Tempo', fontsize=14)
# plt.xlabel('Colunas', fontsize=12)
# plt.ylabel('Valores', fontsize=12)
# plt.xticks(posicoes, estatisticas['Coluna'], rotation=45, ha='right')
# plt.legend()
# plt.grid(axis='y', linestyle='--', alpha=0.7)
# plt.tight_layout()
# plt.show()


### Tempo Medio Janela por Batch

In [ ]:
# mean = batch_df.groupby('batch_id')['tempo_total_batch'].mean()

plt.figure(figsize=(14, 8))
plt.plot(time_df.groupby('batch_id')['tempo_total_janela'].mean(), label='Tempo Medio da Janela')
# plt.fill_between(
#     range(len(conformidade)),
#     time_df.groupby('batch_id')['tempo_total_janela'].min(),
#     time_df.groupby('batch_id')['tempo_total_janela'].max(),
#     alpha=0.3,
#     label='Variação (Min-Max)'
# )
# plt.axhline(y=max_time_per_batch, color='r', linestyle='--', label='Limite de Tempo Real')
plt.title('Tempo Medio Total de cada Janela por Amostra')
plt.xlabel('Batch ID')
plt.ylabel('Tempo (µs)')
plt.legend()
plt.show()

### Tempo Total por Batch

In [ ]:
# mean = batch_df.groupby('batch_id')['tempo_total_batch'].mean()

plt.figure(figsize=(14, 8))
plt.plot(batch_df['tempo_total_batch'], label='Tempo Total Médio por Batch')
# plt.fill_between(
#     range(len(conformidade)),
#     batch_df['tempo_total_batch'].min(),
#     batch_df['tempo_total_batch'].max(),
#     alpha=0.3,
#     label='Variação (Min-Max)'
# )
plt.axhline(y=max_time_per_batch, color='r', linestyle='--', label='Limite de Tempo Real')
plt.title('Tempo Total por Amostra')
plt.xlabel('Batch ID')
plt.ylabel('Tempo (µs)')
plt.legend()
plt.show()

In [ ]:
mean2 = time_df.groupby('batch_id')['tempo_total_batch'].mean()